# Programme Python pour le traitement des données

Compléter la partie du programme spécifiée, pour cela :

   -   Les pointillés sont à remplacer par les valeurs mesurées.
   
   -   Deux mesures sont séparées par une "," et un nombre décimal s'écrit avec un "."
   
   -   Les hauteurs de chute et l'incertitude-type associée à h doivent être exprimées en m. 
       Exemple : h=[0.50,0.42,0.38] et u_h=0.5e-2
       
   -   Les durées de chute et l'incertitude-type associée à Delta_t doivent être exprimées en s.
   
   -   L'interprétation du programme peut prendre une dizaine de seconde... Patience!
   
   -   L'exécution du programme donne : la valeur moyenne de l'intensité de pesanteur, son incertitude-type, l'histogramme des        valeurs de g et la droite moyenne de régression linéaire.

In [ ]:
#*******************************************************************************************#
# Méthode de Monte-Carlo pour le calcul de la pente de la droite de regression linéaire     #
# Application à la détermination de l'intensité de pesanteur terrestre                      #
#*******************************************************************************************#

import sys
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install matplotlib

from scipy.optimize import curve_fit
import numpy as np
from matplotlib import pyplot as plt

# Renvoie une valeur aléatoire de la variable L[0] d'incertitude-type L[1]
def Alea(L):
    tirage=np.random.normal()   # Tirage entre -infini et +infini (loi normale)
    return L[0]+L[1]*tirage

'''*********************Partie du programme à compléter********************'''
# Entrées
h=[...]   # Hauteurs de chute (en m)
u_h=...   # Incertitude type associée à chaque hauteur de chute (en m)

Delta_t=[...]   # Durée de chaque chute (en s)
u_Delta_t=...   # Incertitude type associée à chaque durée de chute (en s)
'''************************************************************************'''

Abs=[]
for k in range(len(Delta_t)):
    Abs.append(0.5*Delta_t[k]**2)

#Formule de propagation des incertitudes appliquée à Abs = 0.5*Delta_t^2
u_Abs=[]
for k in range(len(Delta_t)):
    u_Abs.append(2*Abs[k]*u_Delta_t/Delta_t[k])

# Préparation des listes avec incertitudes
hauteur=[]
for k in range(len(h)):
    hauteur.append([h[k],u_h])

duree=[]
for k in range(len(h)):
    duree.append([Delta_t[k],u_Delta_t])

def lineaire(x,a):
    return a*x

# Méthode de Monte Carlo pour déterminer la pente par régression linéaire
LCoef=[] # Initialisation de la liste vide (coefficient directeur)
iteration=100000

for i in range(iteration):
    Alea_abs=[]
    Alea_h=[]
    for k in range(len(h)):
        Alea_abs.append(0.5*Alea(duree[k])**2)
        Alea_h.append(Alea(hauteur[k]))
    params, covar = curve_fit(lineaire,Alea_abs,Alea_h)
    coef = params[0]
    LCoef.append(coef)

MoyPente=np.mean(LCoef)
uPente=np.std(LCoef, ddof = 1)

print('Pente :',MoyPente,'m.s-2')
print('u(Pente) :',uPente,'m.s-2')

fenetre1 = plt.figure('Pour 100000 iterations')
graph = fenetre1.add_subplot(111)
graph.hist(LCoef, range=(min(LCoef),max(LCoef)), bins = 50, color = 'blue', edgecolor = 'black')
plt.xlabel('Pente (m.s-2)')
plt.ylabel('Effectif')

fenetre2 = plt.figure('h=f(0.5*Delta_t^2)')
plt.plot(Abs,h,'m+',label="Points expérimentaux")
#Tracé des barres d'incertitudes
plt.errorbar(Abs,h,xerr=u_Abs,yerr=u_h,fmt='none',ecolor='blue')
#Tracé de la droite moyenne
X = np.array([min(Abs),max(Abs)])     #Pour tracer une droite, 2 points sont suffisants
plt.plot(X,MoyPente*X,'r-',label="Droite moyenne")
plt.xlabel('0.5*Delta_t^2 (en s^2)')
plt.ylabel('h (en m)')
plt.axis([min(Abs),max(Abs),min(h),max(h)])
plt.title ('h = f(0.5*Delta_t^2)')
plt.text(min(Abs)+0.0050,min(h)+0.025,"Equation de la droite moyenne : h ="+str(round(MoyPente,2))+"*1/2*Delta_t^2",bbox=dict(facecolor='red', alpha=0.5))
plt.legend()
plt.grid()

plt.show()